In [ ]:
import pandas as pd
import sqlite3
from pathlib import Path

In [2]:
# Подключение к базе данных и загрузка всех комментариев
db_path = Path("yt_admin.db")
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row

query = """
SELECT 
    comment_id,
    video_id,
    parent_id,
    text_display,
    text_original,
    author_display_name,
    published_at,
    like_count
FROM comment_details
WHERE text_display IS NOT NULL OR text_original IS NOT NULL
ORDER BY published_at DESC
"""

comments = conn.execute(query).fetchall()
comments_df = pd.DataFrame([dict(row) for row in comments])
conn.close()

print(f"Всего комментариев в базе: {len(comments_df)}")
comments_df.head()

Всего комментариев в базе: 210811


,comment_id,video_id,parent_id,text_display,text_original,author_display_name,published_at,like_count
0,Ugz5DR_y7DyNaxBniTd4AaABAg,VM8DBwcyUXI,None,Всех чиновников под трибунал. Все они коррупци...,Всех чиновников под трибунал. Все они коррупци...,@SNN.2025,2025-11-06T15:32:08Z,0
1,UgxfRWkTbPIX2TY7KNZ4AaABAg,cRFrpHoDGsc,None,Машины старше 20-30 лет сейчас надёжнее новых,Машины старше 20-30 лет сейчас надёжнее новых,@КазаковНиколай-е5ы,2025-11-06T15:25:25Z,0
2,UgxfvrfUBjUCyF6I0YV4AaABAg,sN7zPeGs4R8,None,Народные деньги не народные у народа дыры в ка...,Народные деньги не народные у народа дыры в ка...,@АлександрТайкин-л6у,2025-11-06T15:16:35Z,0
3,UgxEgS2rS9yfsJyk6G94AaABAg.APBY0tzXK_tAPBYvcaV3ae,Q3UT10nBJTI,UgxEgS2rS9yfsJyk6G94AaABAg,"на свой счет в банке, они в доле😂","на свой счет в банке, они в доле😂",@egorsmorgunov8024,2025-11-06T15:08:47Z,0
4,UgwraVv6gM2dBKZ83ed4AaABAg,oaaT9NPBoHQ,None,Assalomu alaykum Sanjar bu ruchkaiz bilan butu...,Assalomu alaykum Sanjar bu ruchkaiz bilan butu...,@abduhalilubadullaev3203,2025-11-06T15:06:49Z,0


In [5]:
# Создаем DataFrame только с колонкой text_display
comments = comments_df[["text_display"]]
comments

,text_display
0,Всех чиновников под трибунал. Все они коррупци...
1,Машины старше 20-30 лет сейчас надёжнее новых
2,Народные деньги не народные у народа дыры в ка...
3,"на свой счет в банке, они в доле😂"
4,Assalomu alaykum Sanjar bu ruchkaiz bilan butu...
...,...
210806,И мы их всех кормим...
210807,Санжар Бокаев можно подсчитать для Азербайджана
210808,"Каждому депутату кайло в руки и в шахту, дават..."
210809,"Проблема не в системе парт строиттельства, а ..."


In [ ]:
# Разделение данных на train (5k), validation (5k) и unseen (остальное)
train_size = 4000
val_size = 2000

# Берем первые 5000 для обучения
train_df = comments_df.head(train_size).copy()

# Следующие 5000 для валидации
val_df = comments_df.iloc[train_size : train_size + val_size].copy()

# Остальное для unseen
unseen_df = comments_df.iloc[train_size + val_size :].copy()

print(f"Train: {len(train_df)}")
print(f"Validation: {len(val_df)}")
print(f"Unseen: {len(unseen_df)}")

Train: 4000
Validation: 2000
Unseen: 204811


In [7]:
# Сохранение в CSV файлы
train_df.to_csv("train.csv", index=False, encoding="utf-8")
val_df.to_csv("validation.csv", index=False, encoding="utf-8")
unseen_df.to_csv("unseen.csv", index=False, encoding="utf-8")

print("Файлы сохранены:")
print("- train.csv")
print("- validation.csv")
print("- unseen.csv")

Файлы сохранены:
- train.csv
- validation.csv
- unseen.csv


In [8]:
comments = pd.concat([train_df, val_df])

In [ ]:
!pip install --upgrade openai


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
import os
import json
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from openai import OpenAI
from tqdm.auto import tqdm

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

/Users/assan/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
SYSTEM_PROMPT = """
Ты модератор. Определи, содержит ли текст ПРИЗЫВ К НАСИЛИЮ.

Призыв = побуждение совершить насилие ("убей", "бей", "взорви", "сожги", "өлтір", "ұр", etc.)

НЕ считать призывом:
- угрозы
- описания
- новости
- сарказм
- агрессию без подстрекательства

ФОРМАТ ОТВЕТА СТРОГО:
Только JSON-массив 0/1.
Пример:
[0,1,0,0,1]
"""

In [12]:
def classify_batch(texts):
    # формируем структурированный JSON
    items = [{"id": i, "text": t} for i, t in enumerate(texts)]
    user_prompt = json.dumps(items, ensure_ascii=False)

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
    )

    raw = resp.choices[0].message.content.strip()

    # вырезаем Markdown и мусор
    cleaned = re.sub(r"```.*?```", "", raw, flags=re.DOTALL)
    cleaned = cleaned.replace("```json", "").replace("```", "").strip()

    # извлекаем JSON
    start = cleaned.find("[")
    end = cleaned.rfind("]") + 1
    json_part = cleaned[start:end]

    try:
        labels = json.loads(json_part)
    except Exception as e:
        print("Ошибка JSON:\n", raw)
        raise e

    return labels

In [13]:
BATCH_SIZE = 8
NUM_WORKERS = 8


def process_batch(idx, batch_texts):
    labels = classify_batch(batch_texts)
    return idx, labels  # возвращаем индекс батча и метки

In [14]:
texts = comments["text_display"].astype(str).tolist()

batches = []
for i in range(0, len(texts), BATCH_SIZE):
    batch_texts = texts[i : i + BATCH_SIZE]
    batches.append((i, batch_texts))  # (стартовый индекс, тексты)

In [15]:
results = {}

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
    futures = []

    for idx, batch_texts in batches:
        futures.append(ex.submit(process_batch, idx, batch_texts))

    for f in tqdm(as_completed(futures), total=len(futures)):
        idx, labels = f.result()
        results[idx] = labels

100%|██████████| 750/750 [02:44<00:00,  4.57it/s]


In [16]:
final_labels = [None] * len(texts)

for idx, labels in results.items():
    for j, val in enumerate(labels):
        final_labels[idx + j] = val

In [17]:
assert None not in final_labels
len(final_labels), len(texts)

AssertionError: 

In [18]:
comments["call_to_violence"] = final_labels

In [ ]:
comments = comments[["text_display", "call_to_violence"]]

In [21]:
comments.to_csv("comments_with_labels.csv", index=False, encoding="utf-8")

In [3]:
import pandas as pd

comments = pd.read_csv("comments_with_labels.csv")

In [4]:
comments.call_to_violence.value_counts()

call_to_violence
0.0    5571
1.0     425
Name: count, dtype: int64

In [5]:
comments = comments[~comments.call_to_violence.isna()]

In [6]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score

/Users/assan/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Убедимся, что нет NaN
comments["call_to_violence"] = comments["call_to_violence"].fillna(0).astype(int)

train_df, temp = train_test_split(
    comments, test_size=0.2, stratify=comments["call_to_violence"], random_state=42
)

val_df, test_df = train_test_split(
    temp, test_size=0.5, stratify=temp["call_to_violence"], random_state=42
)

In [8]:
# Переименуем колонку
train_df = train_df.rename(columns={"call_to_violence": "label"})
val_df = val_df.rename(columns={"call_to_violence": "label"})
test_df = test_df.rename(columns={"call_to_violence": "label"})

In [9]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

In [10]:
MODEL = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_fn(batch):
    return tokenizer(
        batch["text_display"], truncation=True, padding="max_length", max_length=128
    )


train_ds = train_ds.map(tokenize_fn, batched=True)
val_ds = val_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

train_ds = train_ds.remove_columns(
    [
        col
        for col in train_ds.column_names
        if col not in ["input_ids", "attention_mask", "label"]
    ]
)
val_ds = val_ds.remove_columns(
    [
        col
        for col in val_ds.column_names
        if col not in ["input_ids", "attention_mask", "label"]
    ]
)
test_ds = test_ds.remove_columns(
    [
        col
        for col in test_ds.column_names
        if col not in ["input_ids", "attention_mask", "label"]
    ]
)

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")

Map: 100%|██████████| 600/600 [00:00<00:00, 35547.46 examples/s]


In [11]:
pos = train_df["label"].sum()
neg = len(train_df) - pos
class_weights = torch.tensor([1.0, neg / pos], dtype=torch.float)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

# Добавляем веса в loss
model.config.problem_type = "single_label_classification"
model.classifier.loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds), "f1": f1_score(labels, preds)}

In [14]:
from transformers import EarlyStoppingCallback

In [15]:
training_args = TrainingArguments(
    output_dir="./model_violence_ft",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=5, early_stopping_threshold=0.0005
        )
    ],
)

/var/folders/2f/p150q4t507z4w8dhwy3ncd280000gn/T/ipykernel_93333/2062274449.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

/Users/assan/miniconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.247400,0.224748,0.928333,0.000000
2,0.175700,0.180292,0.936667,0.321429


/Users/assan/miniconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/assan/miniconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [45]:
trainer.evaluate(test_ds)

/Users/assan/miniconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.24421676993370056,
 'eval_accuracy': 0.9416666666666667,
 'eval_f1': 0.5569620253164557,
 'eval_runtime': 0.3522,
 'eval_samples_per_second': 1703.432,
 'eval_steps_per_second': 53.942,
 'epoch': 10.0}

In [ ]:
pred = trainer.predict(test_ds)
logits = pred.predictions  # shape [N, 2]
labels = pred.label_ids  # true labels (0/1)

/Users/assan/miniconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
import torch
import numpy as np

probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()[:, 1]  # P(y=1)

threshold = 0.5
preds = (probs >= threshold).astype(int)

In [ ]:
test_df_reset = test_df.reset_index(drop=True)

In [ ]:
df_pred = pd.DataFrame(
    {
        "text": test_df_reset["text_display"],
        "true_label": labels,
        "pred_label": preds,
        "prob_violence": probs,
    }
)

In [ ]:
df_pred[df_pred.true_label == 1]

,text,true_label,pred_label,prob_violence
5,Правильно сделали что уничтожают чуждую нашей ...,1,0,0.009400
8,мы лучше усраину разрушим))),1,0,0.007727
20,Санжар добевай этот утильсбор,1,0,0.002183
23,Надо учиться у Швеции и у Китая все деньги на ...,1,1,0.988734
38,"Не только смертную казнь ввести , прилюдно на ...",1,1,0.983220
51,Возбудить уголовные дела и посадить всех тварей,1,1,0.908162
70,Мля что делать всех поубивать коррупционеров ...,1,1,0.709245
107,Коррупция срослась с прваительством. Нужно нач...,1,1,0.994441
123,"Казнить всех коррупционеров, то есть всех чино...",1,1,0.978364
126,Этих двух индивидуумов надо раскулачить и к ст...,1,1,0.994964
